# Imports and Running the files

In [2]:
%run WordEmbedds.ipynb
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential,Model
from keras.layers import Dense,Bidirectional,LSTM,Embedding, GlobalMaxPooling1D, GlobalMaxPooling3D
from tensorflow.keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint

C:\Users\mangu\AppData\Local\Temp\ipykernel_13244\1802624429.py:49: FutureWarning: The default value of regex will change from True to False in a future version.
  df['processed'] = df['processed'].str.replace('http[^\s]*',"")
C:\Users\mangu\AppData\Local\Temp\ipykernel_13244\1802624429.py:73: FutureWarning: The default value of regex will change from True to False in a future version.
  df['processed'] = df['processed'] .str.replace('[...…]','').str.split().apply(lambda x: ' '.join([replacers.get(e, e) for e in x]))


# Building the Model

In [5]:
#EarlyStopping and ModelCheckpoint to efficiently save the model when there is an improvement in the
#accuracy is high or to stop when the accuracy is not improving further
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
mc = ModelCheckpoint('./model.h5', monitor = 'val_accuracy', mode = 'max', verbose = 1, save_best_only = True)
def build_model(X,Y, classes):
    #Dimension of our embeddings
    model=Sequential()
    #Embedding layer
    model.add(Embedding(input_dim=len(word_index)+1, output_dim=50, input_length=len(X[0]), weights = [embedding_matrix], trainable=False))
    #BiLSTM layers with dropouts
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dense(classes, activation = 'softmax'))
    adam = keras.optimizers.Adam(learning_rate = 0.0001)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    ep = model.fit(X, Y, batch_size=128, epochs=25, verbose = 1, validation_data=(X_valid, y_valid), callbacks= [es, mc])
    return model, ep

In [ ]:
#Building model with only 3 labels
bilstmModel, his=build_model(X_train,y_train, 3)

In [ ]:
#Plotting the training and validation accuracy
plt.plot(his.history['accuracy'],c='b',label='train accuracy')
plt.plot(his.history['val_accuracy'],c='r',label='validation accuracy')
plt.title("BiLSTM model")
plt.legend(loc='lower right')
plt.show()

In [ ]:
#Predicting the model on the test data
y_pred =   np.argmax(bilstmModel.predict(X_test), axis  =  1)
y_true = np.argmax(y_test, axis = 1)
from sklearn import metrics
print(metrics.classification_report(y_pred, y_true))